In [1]:
# Copyright 2022-2023 Sony Semiconductor Solutions Corp. All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Get device list

This notebook explains the workflow for retrieving the list of devices from the "**Console for AITRIOS**". 

Instructions are described in [README.md](./README.md).

## Imports

In [2]:
import pandas as pd
from console_access_library.client import Client
from IPython.display import display

## Load an instance of "**Console Access Library**" client

Run this cell to restore instance variable 'client_obj' created on a notebook of Set up "**Console Access Library**".

In [3]:
%store -r
client_obj: Client = client_obj

## Get device list

In [ ]:
# Get an instance of device management API
device_management_obj = client_obj.get_device_management()

# Call an API for get device list
try:
    response = device_management_obj.get_devices()
except Exception as e:
    # EXCEPTION
    raise e

# response error check
if "result" in response and response["result"] != "SUCCESS":
    # ERROR
    raise ValueError("ERROR", response)

# Create output list
model_lists = []
devices = response.get("devices", [])

for device in devices:
    device_id = device.get("device_id", "")

    model_list = []
    model_list.append(device_id)
    models = device.get("models", [])
    if any(models):
        for model in models:
            version_id = ""
            model_version_id = model.get("model_version_id", "")
            if len(model_version_id) == 0:
                # There is no models deployed on the device
                model_id = ""
            elif ":" not in model_version_id:
                # The model deployed to the device does not exist in Console database
                model_id = model_version_id
            else:
                # Split string to model_id and version_id
                model_version_id_list = model_version_id.split(":v")

                model_id = model_version_id_list[0]
                version_id = model_version_id_list[1]

            model_list.append(model_id)
            model_list.append(version_id)

        # Fill missing elements with empty string
        model_list = model_list + ["" for x in range(9 - len(model_list))]

        model_lists.append(model_list)
    else:
        # There is no models deployed on the device
        model_list.append("")

        # Fill missing elements with empty string
        model_list = model_list + ["" for x in range(9 - len(model_list))]

        model_lists.append(model_list)

if len(model_lists) == 0:
    raise Exception("There is no data in the device list.")

output_frame = pd.DataFrame(
    model_lists, columns=["Device ID", "c1", "c2", "c3", "c4", "c5", "c6", "c7", "c8"]
)

# Specify "Device ID" column as index row
output_frame.set_index("Device ID", inplace=True)

# Set column to 2 rows
double_columns_num = pd.MultiIndex.from_arrays(
    [
        ["Model 1", " ", "Model 2", " ", "Model 3", " ", "Model 4", " "],
        ["ID", "Version", "ID", "Version", "ID", "Version", "ID", "Version"],
    ]
)

# Change column
output_frame.columns = double_columns_num

# setting backup
backup_max_rows = pd.options.display.max_rows
# output limit clear
pd.set_option("display.max_rows", None)
display(output_frame)
# setting restore
pd.set_option("display.max_rows", backup_max_rows)